### **TF-IDF: What Is It?**
TF-IDF is a technique used to measure the relevance of a word within a document in a collection of documents. It combines two metrics:

**Term Frequency (TF):** This represents how many times a word appears in a document.
Inverse Document Frequency (IDF): This accounts for the rarity of a word across the entire collection of documents.
The goal of TF-IDF is to give higher scores to words that are important within a specific document while downplaying common words. It has various applications, including information retrieval, text analysis, keyword extraction, and feature extraction for machine learning algorithms1.

### **How to Compute TF-IDF?**
Let’s break down the steps using an example. Suppose we have the following documents and a query:

**Query (Q): “The cat.”**
- Documents (D1, D2, D3):
    - D1: “The cat is on the mat.”
    - D2: “My dog and cat are the best.”
    - D3: “The locals are playing.”

1. Term Frequency (TF):

We calculate the TF score for each word (query words: “the” and “cat”) in each document. The simplest way is to compute the ratio of the count of instances to the total number of words in the document:

- TF (“the”, D1) = 2/6 = 0.33
- TF (“the”, D2) = 1/7 = 0.14
- TF (“the”, D3) = 1/4 = 0.25
- TF (“cat”, D1) = 1/6 = 0.17
- TF (“cat”, D2) = 1/7 = 0.14
- TF (“cat”, D3) = 0/4 = 0

2. Inverse Document Frequency (IDF):

We calculate the IDF score for each word:
- IDF (“the”) = log(3/3) = log(1) = 0
- IDF (“cat”) = log(3/2) = 0.18

3. TF-IDF Score:

Finally, we multiply the TF and IDF scores to get the TF-IDF score for each word in each document:
- TF-IDF (“the”, D1) = 0.33 * 0 = 0
- TF-IDF (“the”, D2) = 0.14 * 0 = 0
- TF-IDF (“the”, D3) = 0.25 * 0 = 0
- TF-IDF (“cat”, D1) = 0.17 * 0.18 = 0.0306
- TF-IDF (“cat”, D2) = 0.14 * 0.18 = 0.0252


The higher the TF-IDF score, the more relevant the word is in that specific document1.

**Applications:**
- Text Vectorization: TF-IDF is often used to transform text into numeric vectors, representing the content of the text.
- Information Retrieval: Helps find relevant documents based on queries.
- Machine Learning Features: Provides features for ML models.

# Naive Bayes

In [ ]:
import os
import io
from pandas import DataFrame
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

import warnings
warnings.filterwarnings("ignore")

In [21]:
def readFiles(path):
    for root, dirnames, filenames in os.walk(path):
        for filename in filenames:
            path = os.path.join(root, filename)

            f = io.open(path, 'r', encoding='latin1')
            inBody = False
            lines = []
            for line in f:
                if inBody:
                    lines.append(line)
                elif line == '\n':
                    inBody = True
            f.close()
            message = '\n'.join(lines)
            yield path, message

In [22]:
def dataFrameFromDirectory(path, classification):
    rows = []
    index = []
    for filename, message in readFiles(path):
        rows.append({'message': message, 'class': classification})
        index.append(filename)

    return DataFrame(rows, index=index)

In [43]:
data = DataFrame({'message': [], 'class': []})

data = pd.concat([data, dataFrameFromDirectory(r"D:\Machine learning\Workbook\Naive Bayes\spam", 'spam')])
data = pd.concat([data, dataFrameFromDirectory(r"D:\Machine learning\Workbook\Naive Bayes\ham", 'ham')])

In [44]:
data.head()

,message,class
D:\Machine learning\Workbook\Naive Bayes\spam\00001.7848dde101aa985090474a91ec93fcf0,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 Tr...",spam
D:\Machine learning\Workbook\Naive Bayes\spam\00002.d94f1b97e48ed3b553b3508d116e6a09,1) Fight The Risk of Cancer!\n\nhttp://www.adc...,spam
D:\Machine learning\Workbook\Naive Bayes\spam\00003.2ee33bc6eacdb11f38d052c44819ba6c,1) Fight The Risk of Cancer!\n\nhttp://www.adc...,spam
D:\Machine learning\Workbook\Naive Bayes\spam\00004.eac8de8d759b7e74154f142194282724,##############################################...,spam
D:\Machine learning\Workbook\Naive Bayes\spam\00005.57696a39d7d84318ce497886896bf90d,I thought you might like these:\n\n1) Slim Dow...,spam


In [45]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2983 entries, D:\Machine learning\Workbook\Naive Bayes\spam\00001.7848dde101aa985090474a91ec93fcf0 to D:\Machine learning\Workbook\Naive Bayes\ham\02500.05b3496ce7bca306bed0805425ec8621
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   message  2983 non-null   object
 1   class    2983 non-null   object
dtypes: object(2)
memory usage: 69.9+ KB


In [46]:
data = data.replace(['ham','spam'],[0, 1])
data.head(10)

,message,class
D:\Machine learning\Workbook\Naive Bayes\spam\00001.7848dde101aa985090474a91ec93fcf0,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 Tr...",1
D:\Machine learning\Workbook\Naive Bayes\spam\00002.d94f1b97e48ed3b553b3508d116e6a09,1) Fight The Risk of Cancer!\n\nhttp://www.adc...,1
D:\Machine learning\Workbook\Naive Bayes\spam\00003.2ee33bc6eacdb11f38d052c44819ba6c,1) Fight The Risk of Cancer!\n\nhttp://www.adc...,1
D:\Machine learning\Workbook\Naive Bayes\spam\00004.eac8de8d759b7e74154f142194282724,##############################################...,1
D:\Machine learning\Workbook\Naive Bayes\spam\00005.57696a39d7d84318ce497886896bf90d,I thought you might like these:\n\n1) Slim Dow...,1
D:\Machine learning\Workbook\Naive Bayes\spam\00006.5ab5620d3d7c6c0db76234556a16f6c1,A POWERHOUSE GIFTING PROGRAM You Don't Want To...,1
D:\Machine learning\Workbook\Naive Bayes\spam\00007.d8521faf753ff9ee989122f6816f87d7,Help wanted. We are a 14 year old fortune 500...,1
D:\Machine learning\Workbook\Naive Bayes\spam\00008.dfd941deb10f5eed78b1594b131c9266,<html>\n\n<head>\n\n<title>ReliaQuote - Save U...,1
D:\Machine learning\Workbook\Naive Bayes\spam\00009.027bf6e0b0c4ab34db3ce0ea4bf2edab,TIRED OF THE BULL OUT THERE?\n\nWant To Stop L...,1
D:\Machine learning\Workbook\Naive Bayes\spam\00010.445affef4c70feec58f9198cfbc22997,"Dear ricardo1 ,\n\n\n\n<html>\n\n<body>\n\n<ce...",1


In [49]:
data['class'].value_counts()

class
0    2483
1     500
Name: count, dtype: int64

In [50]:
X = data['message']
y  = data['class']

In [51]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, train_size = 0.8, random_state = 42)

In [53]:
len(x_train)

2386

So we have 597 messages for testing, and 2386 messages for training!

In [54]:
vectorizer = CountVectorizer()
counts = vectorizer.fit_transform(x_train)

classifier = MultinomialNB()
targets = y_train
classifier.fit(counts, targets)

MultinomialNB()

In [55]:
y_pred = classifier.predict(vectorizer.transform(x_test))    
y_pred                              

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,

In [56]:
# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)*100
print(f"Accuracy: {accuracy:.2f} %")

Accuracy: 94.81 %


In [18]:
classification_report = classification_report(y_test, y_pred)
print("Classification Report:")
print(classification_report)

Classification Report:
              precision    recall  f1-score   support

           0       0.94      1.00      0.97       480
           1       0.99      0.74      0.85       117

    accuracy                           0.95       597
   macro avg       0.96      0.87      0.91       597
weighted avg       0.95      0.95      0.95       597



## **Example**

In [58]:
examples = ["$$DON'T WAIT!!! CALL NOW $$ 1-800-896-6568", "Hi Bob, how about a game of golf tomorrow?"]
example_counts = vectorizer.transform(examples)
predictions = classifier.predict(example_counts)
predictions[:2]

array([1, 0])